## Prepare data reader and model execution context

In [1]:
from pprint import pprint

from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.coreutils.partitioned.reader import DataReader
from op_analytics.datapipeline.etl.ingestion.reader.byblock import construct_readers_byblock
from op_analytics.datapipeline.etl.ingestion.reader.request import BlockBatchRequest
from op_analytics.datapipeline.models.compute.markers import ModelsDataSpec
from op_analytics.datapipeline.models.compute.testutils import setup_execution_context

model_name = "refined_traces"


# Prepare data raeders
data_spec = ModelsDataSpec(models=[model_name])
blockbatch_request = BlockBatchRequest.build(
    chains=["op"],
    range_spec="@20241118:+1",
    root_paths_to_read=data_spec.input_root_paths,
)
readers: list[DataReader] = construct_readers_byblock(
    blockbatch_request=blockbatch_request,
    read_from=DataLocation.GCS,
)


# Show details for the batch we are processing.
pprint(readers[0])

# Set up execution context and get handles to model input args.
# In subsequent cells you can use the model input args however you want.
ctx, input_datasets, auxiliary_templates = setup_execution_context(
    model_name=model_name,
    data_reader=readers[0],  # use the first reader
)


2025-01-10 04:14:58 [debug    ] connecting to GOLDSKY Clickhouse client... counter=001/001 eta=None filename=client.py lineno=36 process=2658
2025-01-10 04:14:58 [info     ] loaded vault from .env file    counter=001/001 eta=None filename=vault.py lineno=32 process=2658
2025-01-10 04:14:58 [debug    ] loaded vault: 18 items         counter=001/001 eta=None filename=vault.py lineno=76 process=2658
2025-01-10 04:14:59 [debug    ] initialized GOLDSKY Clickhouse client. counter=001/001 eta=None filename=client.py lineno=41 process=2658
2025-01-10 04:14:59 [debug    ] connecting to OPLABS Clickhouse client... filename=client.py lineno=36 process=2658
2025-01-10 04:15:00 [debug    ] initialized OPLABS Clickhouse client. filename=client.py lineno=41 process=2658
2025-01-10 04:15:00 [info     ] prepared 22 input batches.     filename=byblock.py lineno=78 process=2658
DataReader(partitions=Partition(cols=[PartitionColumn(name='chain', value='op'),
                                      Partition

## Execute the model

It's up to you how the model manipulates the data. 

Develop the various steps in notebook cells and then copy the final code over to the model function.

In [2]:
refined_txs_table = auxiliary_templates["refined_transactions_fees"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "raw_blocks": input_datasets["ingestion/blocks_v1"].as_subquery(),
        "raw_transactions": input_datasets["ingestion/transactions_v1"].as_subquery(),
    }
)

refined_traces_projection = auxiliary_templates["refined_traces/traces_projection"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "raw_traces": input_datasets["ingestion/traces_v1"].as_subquery(),
    },
)

traces_with_gas_used = auxiliary_templates["refined_traces/traces_with_gas_used"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "refined_traces_projection": refined_traces_projection,
    },
)

traces_txs_join = auxiliary_templates["refined_traces/traces_txs_join"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "traces_with_gas_used": traces_with_gas_used,
        "refined_transactions_fees": refined_txs_table,
    },
)

2025-01-10 04:15:26 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=245 process=2658
2025-01-10 04:15:26 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=245 process=2658
2025-01-10 04:15:26 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=2658 template=refined_transactions_fees
2025-01-10 04:15:31 [info     ] duck db size: 8.4MB            filename=client.py lineno=36 process=2658
2025-01-10 04:15:31 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=245 process=2658
2025-01-10 04:15:31 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=2658 template=refined_traces/traces_projection
2025-01-10 04:15:35 [info     ] duck db size: 59.0MB           filename=client.py lineno=36 process=2658
2025-01-10 04:15:35 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=2658 template=refin

In [5]:
ctx.client.sql("""
SELECT * FROM refined_traces__traces_txs_join
WHERE transaction_hash = '0x99d4d4dea9419810d61e1899565d10a0f6c7d284e7dcf0e01eff5025833fb0c6'
ORDER BY trace_address
LIMIT 10
""").show(max_rows=100)

┌────────────┬─────────┬──────────┬─────────┬──────────────┬─────────────────┬────────────────────────────────────────────────────────────────────┬───────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬─────────────────┬────────────────┬───────────────┬────────────┬──────────────┬─────────┬─────────────────┬───────────────┬─────────────┬──────────────────────┬───────────────────┬────────────────────────┬──────────────────────────┬──────────────────────────────────┬───────────────────────────────────────┬───────────────────────────────────────────┬──────────────────────────────────────────────┬──────────────────────────────────┬──────────────────────────────────────────┬──────────────────────────────────────────┬──────────────────────────────────────────┬─────────────────────────────────────┬─────────────────────────────────┬────────────────────────────────────┬────────────────────────────────────┬───────────────────────────────────

In [6]:
ctx.client.sql(f"""DESCRIBE {refined_txs_table}""").show(max_rows=100)

┌─────────────────────────────────────┬────────────────┬─────────┬─────────┬─────────┬─────────┐
│             column_name             │  column_type   │  null   │   key   │ default │  extra  │
│               varchar               │    varchar     │ varchar │ varchar │ varchar │ varchar │
├─────────────────────────────────────┼────────────────┼─────────┼─────────┼─────────┼─────────┤
│ dt                                  │ DATE           │ YES     │ NULL    │ NULL    │ NULL    │
│ chain                               │ VARCHAR        │ YES     │ NULL    │ NULL    │ NULL    │
│ chain_id                            │ INTEGER        │ YES     │ NULL    │ NULL    │ NULL    │
│ network                             │ VARCHAR        │ YES     │ NULL    │ NULL    │ NULL    │
│ nonce                               │ BIGINT         │ YES     │ NULL    │ NULL    │ NULL    │
│ transaction_index                   │ BIGINT         │ YES     │ NULL    │ NULL    │ NULL    │
│ from_address                